In [44]:
# instruciton datasetを今回のSFT形式に変換する

In [25]:
import os

#dataフォルダ内をリセット
os.system("rm -rf data/*")

0

In [26]:
from datasets import load_dataset
import json

# ichikara

In [27]:
# 理研のichikara dataset (CC-NC-NDライセンス)
dataset=load_dataset("p1atdev/ichikara-instruction",'20231221-003')["train"]

In [28]:
print(len(dataset))

1899


In [29]:
"""
本番はこっち｡
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい
"""

#question_template="### 指示：以下の質問に答えなさい。 ### 質問："
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"

In [30]:
n_instructions=1500
ichikara_list=[]

for n_instructions in [1500]:
    output_path=f"data/ichikara_{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(dataset)
    for i in range(n_instructions+100):
        original_record=next(loader)
        q=original_record["text"]
        a=original_record["output"]
        text=f"{question_template}{q}{answer_template}{a}"

        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            ichikara_list.append(line)
 
        with open(output_path+eval_prefix,"a") as f:
            line=json.dumps({"text":text},ensure_ascii=False)
            f.write(line+"\n")


ichikara_list=list(set(ichikara_list))

# dolly

In [31]:
d_dataset=load_dataset("kunishou/databricks-dolly-15k-ja")["train"]

In [32]:
d_dataset[0]

{'category': 'closed_qa',
 'input': 'ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。',
 'instruction': 'ヴァージン・オーストラリア航空はいつから運航を開始したのですか？',
 'index': '0',
 'output': 'ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しました。'}

In [33]:
#question_template2="### 指示：以下の質問に答えなさい。"
#question_template2="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい｡"
#question_template2="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 質問:\n"
dolly_list=[]
for n_instructions in [10000]:
    output_path=f"data/dolly{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(d_dataset)
    for i in range(n_instructions+100):
        original_record=next(loader)
        if "input" in original_record:
            inp=original_record["input"]
        else:
            inp=""
        q=original_record["instruction"]
        a=original_record["output"]
        if inp=="":
            text=f"{question_template}{q}{answer_template}{a}"
        else:
            text=f"{question_template}{q}\n\n### 入力:\n{inp}\n{answer_template}{a}"
        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            dolly_list.append(line)
        with open(output_path+eval_prefix,"a") as f:
            line=json.dumps({"text":text},ensure_ascii=False)
            f.write(line+"\n")

dolly_list=list(set(dolly_list))

# oasst

In [34]:
oasst = load_dataset("kunishou/oasst1-chat-44k-ja")["train"]

In [35]:

n_instructions=1500
oasst_list=[]

for n_instructions in [40000]:
    output_path=f"data/oasst{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(oasst)
    for i in range(n_instructions+100):
        original_record=next(loader)["conversations"]
        q2=""
        a2=""
        q=original_record[0]["value"]
        a=original_record[1]["value"]

        if len(original_record)==4:
            q2=original_record[2]["value"]
            a2=original_record[3]["value"]

            text=f"{question_template}{q}{answer_template}{a} ### 指示:\n{q2}\n\n### 応答:\n{a2}"
        else:
            text=f"{question_template}{q}{answer_template}{a}"

        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            oasst_list.append(line)
 
        with open(output_path+eval_prefix,"a") as f:
            line=json.dumps({"text":text},ensure_ascii=False)
            f.write(line+"\n")


oasst_list=list(set(oasst_list))

In [36]:
#両方
"""
n_ichi=1500
n_dolly=8000
output_path=f"data/dolly_{n_dolly}_ichi{n_ichi}.jsonl"
with open(output_path,"w") as f:
    f.write("")
for i in range(n_dolly):
    with open(output_path,"a") as f:
        line=json.dumps({"text":dolly_list[i]},ensure_ascii=False)
        f.write(line+"\n")
for i in range(n_ichi):
    with open(output_path,"a") as f:
        line=json.dumps({"text":ichikara_list[i]},ensure_ascii=False)
        f.write(line+"\n")

"""
#両方
n_ichi=1500
n_dolly=8000
n_oasst=35000
output_path=f"data/dolly_{n_dolly}_oasst_{n_oasst}_ichi{n_ichi}.jsonl"
with open(output_path,"w") as f:
    f.write("")

for i in range(n_dolly):
    with open(output_path,"a") as f:
        line=json.dumps({"text":dolly_list[i]},ensure_ascii=False)
        f.write(line+"\n")
for i in range(n_oasst):
    with open(output_path,"a") as f:
        line=json.dumps({"text":oasst_list[i]},ensure_ascii=False)
        f.write(line+"\n")
for i in range(n_ichi):
    with open(output_path,"a") as f:
        line=json.dumps({"text":ichikara_list[i]},ensure_ascii=False)
        f.write(line+"\n")




In [37]:
import glob
eval_path_list=glob.glob("data/*.eval")
eval_data=[]
for eval_path in eval_path_list:
    with open(eval_path) as f:
        line=f.readline()
        eval_data.append(line.strip())
with open(output_path+".eval","w") as f:
    for line in eval_data:
        f.write(line+"\n")

# llm_jap
- https://huggingface.co/datasets/izumi-lab/llm-japanese-dataset
- 2gbもあってでかい

In [39]:

llm_jap= load_dataset("izumi-lab/llm-japanese-dataset", revision="main")["train"]

KeyboardInterrupt: 

In [ ]:

llm_jap_list=[]
for n_instructions in [9074000]:
    output_path=f"data/llm_jap_{n_instructions}.jsonl"
    with open(output_path,"w") as f:
        f.write("")
    loader=iter(d_dataset)
    for i in range(n_instructions+100):
        original_record=next(loader)
        if "input" in original_record:
            inp=original_record["input"]
        else:
            inp=""
        q=original_record["instruction"]
        a=original_record["output"]
        if inp=="":
            text=f"{question_template}{q}{answer_template}{a}"
        else:
            text=f"{question_template}{q}\n\n### 入力:\n{inp}\n{answer_template}{a}"
        eval_prefix=""
        if i>=n_instructions:
            eval_prefix=".eval"
        else:
            llm_jap_list.append(line)
        with open(output_path+eval_prefix,"a") as f:
            line=json.dumps({"text":text},ensure_ascii=False)
            f.write(line+"\n")

llm_jap_list=list(set(llm_jap_list))